In [1]:
import pandas as pd 

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


<b>Data Description:</b><br>
Data Source:<br>
- This data is acquired from https://docs.google.com/spreadsheets/d/1caaR9pT24GNmq3rDQpMiIMJrmiTGarbs/edit?gid=1150341366#gid=1150341366 as .xlsx file and converted to data.csv file.<br>
- This dataset contains the following columns:<br>
1.	Id:	To count the records.<br>
2.	MSSubClass:	 Identifies the type of dwelling involved in the sale.<br>
3.	MSZoning:	Identifies the general zoning classification of the sale.<br>
4.	LotArea:	 Lot size in square feet.<br>
5.	LotConfig:	Configuration of the lot<br>
6.	BldgType:	Type of dwelling<br>
7.	OverallCond:	Rates the overall condition of the house<br>
8.	YearBuilt:	Original construction year<br>
9.	YearRemodAdd:	Remodel date (same as construction date if no remodeling or additions).<br>
11.	BsmtFinSF2:	Type 2 finished square feet.<br>
12.	TotalBsmtSF:	Total square feet of basement area<br>
13.	SalePrice:	To be predicted<br>
<b>Data Question:</b><br>
- What factors are significant in predciting sale price of houses? And what price is the house based on given factors's information?<br>
The response variable: SalePrice<br>
The possible predictors:  MSSubClass, MSZoning, LotArea, LotConfig, BldgType, OverallCond, YearBuilt, YearRemodAdd, BsmtFinSF2, TotalBsmtSF<br>


In [2]:
# Load and show the first 10 rows of the data to see data's structure
df = pd.read_csv('data.csv')
df.head(10)

,Id,MSSubClass,MSZoning,LotArea,LotConfig,BldgType,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,BsmtFinSF2,TotalBsmtSF,SalePrice
0,0,60,RL,8450,Inside,1Fam,5,2003,2003,VinylSd,0.0,856.0,208500.0
1,1,20,RL,9600,FR2,1Fam,8,1976,1976,MetalSd,0.0,1262.0,181500.0
2,2,60,RL,11250,Inside,1Fam,5,2001,2002,VinylSd,0.0,920.0,223500.0
3,3,70,RL,9550,Corner,1Fam,5,1915,1970,Wd Sdng,0.0,756.0,140000.0
4,4,60,RL,14260,FR2,1Fam,5,2000,2000,VinylSd,0.0,1145.0,250000.0
5,5,50,RL,14115,Inside,1Fam,5,1993,1995,VinylSd,0.0,796.0,143000.0
6,6,20,RL,10084,Inside,1Fam,5,2004,2005,VinylSd,0.0,1686.0,307000.0
7,7,60,RL,10382,Corner,1Fam,6,1973,1973,HdBoard,32.0,1107.0,200000.0
8,8,50,RM,6120,Inside,1Fam,5,1931,1950,BrkFace,0.0,952.0,129900.0
9,9,190,RL,7420,Corner,2fmCon,6,1939,1950,MetalSd,0.0,991.0,118000.0


In [3]:
# Check data type of each variable an correct the data type
df.info()
# Count categorical columns
categorical_columns = df.select_dtypes(include=['object', 'category', 'bool'])
print(f'Number of categorical columns: {len(categorical_columns.columns)}')
print(categorical_columns.columns)

# Count numeric columns:
numeric_columns = df.select_dtypes(include=['number'])
print(f'Number of numeric columns: {len(numeric_columns.columns)}')
print(numeric_columns.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            2919 non-null   int64  
 1   MSSubClass    2919 non-null   int64  
 2   MSZoning      2915 non-null   object 
 3   LotArea       2919 non-null   int64  
 4   LotConfig     2919 non-null   object 
 5   BldgType      2919 non-null   object 
 6   OverallCond   2919 non-null   int64  
 7   YearBuilt     2919 non-null   int64  
 8   YearRemodAdd  2919 non-null   int64  
 9   Exterior1st   2918 non-null   object 
 10  BsmtFinSF2    2918 non-null   float64
 11  TotalBsmtSF   2918 non-null   float64
 12  SalePrice     1460 non-null   float64
dtypes: float64(3), int64(6), object(4)
memory usage: 296.6+ KB
Number of categorical columns: 4
Index(['MSZoning', 'LotConfig', 'BldgType', 'Exterior1st'], dtype='object')
Number of numeric columns: 9
Index(['Id', 'MSSubClass', 'LotArea', 'Overall

- This dataset has 2919 entries and 13 columns.<br>
- That includes:
4 categorical variables: 'MSZoning', 'LotConfig', 'BldgType', 'Exterior1st'<br>
9 numeric varibles: 'Id', 'MSSubClass', 'LotArea', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF2', 'TotalBsmtSF', 'SalePrice'

In [4]:
# Drop Id column
if 'Id' in df.columns:
    df.drop('Id', axis=1, inplace=True)

# Check data statistic of numeric variables
df.describe()

,MSSubClass,LotArea,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF2,TotalBsmtSF,SalePrice
count,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2918.000000,2918.000000,1460.000000
mean,57.137718,10168.114080,5.564577,1971.312778,1984.264474,49.582248,1051.777587,180921.195890
std,42.517628,7886.996359,1.113131,30.291442,20.894344,169.205611,440.766258,79442.502883
min,20.000000,1300.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,34900.000000
25%,20.000000,7478.000000,5.000000,1953.500000,1965.000000,0.000000,793.000000,129975.000000
50%,50.000000,9453.000000,5.000000,1973.000000,1993.000000,0.000000,989.500000,163000.000000
75%,70.000000,11570.000000,6.000000,2001.000000,2004.000000,0.000000,1302.000000,214000.000000
max,190.000000,215245.000000,9.000000,2010.000000,2010.000000,1526.000000,6110.000000,755000.000000


In [5]:
# Check missing/null values of data
df.isnull().sum()

MSSubClass         0
MSZoning           4
LotArea            0
LotConfig          0
BldgType           0
OverallCond        0
YearBuilt          0
YearRemodAdd       0
Exterior1st        1
BsmtFinSF2         1
TotalBsmtSF        1
SalePrice       1459
dtype: int64

- MSZoning has 4 null values<br>
- Exterior1st has 1 null values<br>
- BsmtFinSF2 has 1 null values<br>
- TotalBsmtSF has 1 null values<br>
- SalePrice has 1459 null values<br>

In [6]:
# Replace null values with mean values and remove duplicate rows if any in the dataset
df. drop_duplicates(inplace=True)

# Fill na for categorical variables
df['MSZoning'].fillna(df['MSZoning'].mode()[0], inplace = True)
df['Exterior1st'].fillna(df['Exterior1st'].mode()[0], inplace = True)

# Fill na for numeric variables
df['BsmtFinSF2'].fillna(df['BsmtFinSF2'].mean(), inplace = True)
df['TotalBsmtSF'].fillna(df['TotalBsmtSF'].mean(), inplace = True)
df['SalePrice'].fillna(df['SalePrice'].mean(), inplace = True)
df.shape

(2911, 12)

After cleaning:<br>
- This dataset has 2911 entries and 12 columns.<br>
- That includes:
4 categorical variables: 'MSZoning', 'LotConfig', 'BldgType', 'Exterior1st'<br>
9 numeric varibles: 'MSSubClass', 'LotArea', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF2', 'TotalBsmtSF', 'SalePrice'

In [7]:
# Plot bar chart to see the distribution of each variable
value_count = df['MSZoning'].value_counts().reset_index()
value_count.columns = ['Category', 'Count']
plot = px.bar(value_count, x='Category', y='Count', title='MSZoning Category Distribution')
pyo.iplot(plot)

Most Common Zoning Classification:<br>
"RL" (Residential Low Density): This is the most common zoning classification by a wide margin, with a count of 2267. This suggests that the majority of properties in the dataset fall under low-density residential zoning, typically indicating larger lots or single-family homes.

Less Common Zoning Classifications:<br>
"RM" (Residential Medium Density): The next most common classification is RM, with 456 properties. This suggests a significant but much smaller portion of the dataset is designated for medium-density residential use, which could include smaller lots or multi-family housing.

Rare Zoning Classifications:<br>
"FV" (Floating Village Residential): This category has 137 properties, making it less common but still present in the dataset. FV zoning is likely specialized and uncommon.

"RH" (Residential High Density): This category, with only 26 properties, indicates that high-density residential areas are rare in the dataset.
"C (all)" (Commercial): The commercial zoning category has the lowest count at 25, suggesting that commercial properties are very rare compared to residential ones.

Conclusion:<br>
The chart illustrates a strong prevalence of low-density residential zoning (RL) in the dataset, with medium-density residential zoning (RM) as a distant second. High-density residential (RH), Floating Village Residential (FV), and Commercial zoning categories (C) are much less common, suggesting that the dataset is predominantly focused on residential areas, particularly those with lower density. This distribution could reflect suburban or rural characteristics, where large, single-family homes are more typical.

In [8]:
value_count = df['LotConfig'].value_counts().reset_index()
value_count.columns = ['Category', 'Count']
plot = px.bar(value_count, x='Category', y='Count', title='LotConfig Category Distribution')
pyo.iplot(plot)

Most Common Lot Configuration:<br>
"Inside": The "Inside" lot configuration is the most common, with a count of 2125. This suggests that most properties in the dataset have this type of lot configuration, which is likely a typical interior lot without special frontage or corner exposure.

Less Common Configurations:<br>
"Corner": The "Corner" configuration is the next most common, with a count of 511. Corner lots often have more street frontage and may be more desirable in certain contexts.

"CulDSac": Cul-de-Sac lots have a count of 176, indicating that they are less common but still significant. These lots are typically located at the end of a dead-end street and may offer more privacy or reduced traffic.

Rare Configurations:<br>
"FR2": Frontage Type 2 has a low count of 85, making it one of the less common configurations.

"FR3": Frontage Type 3 is the rarest configuration in the dataset, with only 14 occurrences. This suggests that this particular frontage type is uncommon.

Conclusion:<br>
The chart indicates a clear dominance of "Inside" lot configurations in the dataset, with corner lots being the second most common. Cul-de-Sac lots are present but less frequent, while Frontage Types 2 and 3 are quite rare. This distribution could reflect common lot configurations in residential areas, with most properties being standard interior lots, while specialized lot types like corner lots, Cul-de-Sac, and various frontage types are less frequent.

In [9]:
value_count = df['BldgType'].value_counts().reset_index()
value_count.columns = ['Category', 'Count']
plot = px.bar(value_count, x='Category', y='Count', title='BldgType Category Distribution')
pyo.iplot(plot)

Most Common Building Type:

"1Fam" (Single-Family Detached Homes): This category is by far the most common, with a count approaching 2500. This suggests that single-family detached homes dominate the dataset.

Other Building Types:<br>
"TwnhsE" (Townhouse End Unit): This is the next most common building type, but with a significantly lower count compared to "1Fam," at around 200.

"Duplex": Duplexes have a count of just above 100, indicating they are relatively uncommon in the dataset.

"Twnhs" (Townhouse Inside Unit): Townhouses (inside units) also have a low count, similar to Duplexes, indicating limited presence in the dataset.

"2fmCon" (Two-Family Conversion; i.e., a single-family home converted into two units): This category has the lowest count, close to that of Townhouses and Duplexes, making it one of the least common building types.

Conclusion:<br>
The chart reveals a strong preference for single-family detached homes ("1Fam"), which vastly outnumbers the other building types. Townhouses, duplexes, and two-family conversions are much less common, suggesting that the dataset may be focused on a suburban or single-family residential area where detached homes are the standard.

In [10]:
value_count = df['Exterior1st'].value_counts().reset_index()
value_count.columns = ['Category', 'Count']
plot = px.bar(value_count, x='Category', y='Count', title='Exterior1st Category Distribution')
pyo.iplot(plot)

Most Common Exterior Material:

"VinylSd": Vinyl Siding (VinylSd) is the most common exterior material, with a count exceeding 1000. This suggests that it is the preferred or most frequently used exterior covering in the dataset.
Other Common Materials:

"MetalSd" (Metal Siding), "HdBoard" (Hardboard), "Wd Sdng" (Wood Siding): These materials have moderately high counts, each ranging between 400 to 500. They are also popular choices but are less common than Vinyl Siding.
"Plywood": Plywood also has a significant presence, with a count just under 300.
Less Common Materials:

"CemntBd" (Cement Board), "BrkFace" (Brick Face): These materials have lower counts, around 100-150, indicating that they are less commonly used compared to the materials mentioned above.
"WdShing" (Wood Shingles), "AsbShng" (Asbestos Shingles), "Stucco": These materials have even lower counts, under 100.
Least Common Materials:

"BrkComm" (Brick Common), "AsphShn" (Asphalt Shingles), "Stone", "CBlock" (Concrete Block), "ImStucc" (Imitation Stucco): These materials have the lowest counts, barely visible on the chart, suggesting they are rare or scarcely used.
Conclusion:
The plot indicates a clear preference for Vinyl Siding as the exterior material in the dataset, with Metal Siding, Hardboard, and Wood Siding also being popular. Other materials like Cement Board and Brick Face are used but to a lesser extent, while materials like Stone, Asphalt Shingles, and Concrete Block are the least common.

In [11]:

# Plot Numerical Variables
fig = px.histogram(df, x='MSSubClass', nbins=30, title='Histogram of MSSubClass')
# Customize the appearance if needed
fig.update_layout(
    xaxis_title='MSSubClass',
    yaxis_title='Counts',
    bargap=0.1,  # Adjust the gap between bars
    title_x=0.5   # Center the title
    )
pyo.iplot(fig)

Key Observations:<br>
Distribution:<br>
The histogram visualizes how frequently each MSSubClass category appears in the dataset. The x-axis lists numerous categories (like 20, 60, 70, etc.), and the y-axis shows how many instances of each category are present.

Categories such as 20, 60, and 120 are likely to have higher counts, meaning they are more common dwelling types within the dataset.
Less common classes (e.g., 85, 190) may appear with lower counts, indicating these types of dwellings are rare in the dataset.

Histogram Characteristics:<br>
The histogram is unstacked and has a small bargap (0.1), which means the bars are closely spaced, providing a continuous view of the distribution.

This layout allows for easy comparison between the frequencies of different dwelling classes.

Conclusion:<br>
The histogram offers a visual overview of the distribution of dwelling types (as categorized by MSSubClass) in the dataset. By examining this chart, you can quickly determine which dwelling types are most and least common. This distribution might help in understanding the housing stock within the dataset, identifying the prevalence of certain property types, or detecting any biases or gaps in the data regarding dwelling classifications.

In [12]:
# Plot Numerical Variables
fig = px.histogram(df, x='LotArea', nbins=30, title='Histogram of LotArea')
# Customize the appearance if needed
fig.update_layout(
    xaxis_title='LotArea',
    yaxis_title='Counts',
    bargap=0.1,  # Adjust the gap between bars
    title_x=0.5   # Center the title
    )
pyo.iplot(fig)

Key Observations:<br>
Distribution:<br>
The histogram visualizes the frequency of properties with different lot sizes. The x-axis lists various lot sizes (e.g., 8450, 9600, 11250, etc.), and the y-axis shows how many properties have these specific lot sizes.
Lot sizes appear to vary greatly, from smaller lots around a few thousand square feet to much larger ones exceeding 50,000 square feet.

Histogram Characteristics:<br>
The histogram uses a binning strategy with nbinsx set to 30, which divides the LotArea into 30 bins. This helps to visualize the distribution more clearly, even if the lot sizes range widely.

A small bargap of 0.1 indicates that the bars are closely spaced, providing a continuous view of the distribution.
The plot is unstacked and each bar corresponds to the number of properties within that particular lot size range.

Conclusion:<br>
The histogram offers a visual overview of the distribution of lot sizes in the dataset. This type of visualization is particularly useful for identifying common lot sizes, understanding the spread of lot areas, and detecting outliers or unusually large or small lots. The distribution might help in understanding the nature of the properties within the dataset, such as whether most properties are on small, medium, or large lots. This insight can be critical for real estate analysis, property valuation, and urban planning studies.

In [13]:
# Plot Numerical Variables
fig = px.histogram(df, x='OverallCond', nbins=30, title='Histogram of OverallCond')
# Customize the appearance if needed
fig.update_layout(
    xaxis_title='OverallCond',
    yaxis_title='Counts',
    bargap=0.1,  # Adjust the gap between bars
    title_x=0.5   # Center the title
    )
pyo.iplot(fig)

Key Observations:<br>
Distribution:<br>

The histogram visualizes the frequency of properties with different overall condition ratings. The values of "OverallCond" on the x-axis indicate the ratings, with most of the properties likely clustering around certain ratings.

For instance, a rating of "5" might be the most common, representing average condition, with fewer properties rated as "1" or "9."

Histogram Characteristics:<br>
The histogram uses a binning strategy with nbinsx set to 30, though given that "OverallCond" likely contains a limited range of integer values, the histogram bars likely correspond directly to each rating.

The plot is unstacked and each bar represents the number of properties that received a specific condition rating.

Conclusion:<br>
The histogram provides a visual summary of how the properties in the dataset are rated in terms of their overall condition. This type of analysis is useful for understanding the general quality of properties within the dataset, identifying common ratings, and potentially spotting any skew towards higher or lower quality properties. It could be particularly informative when comparing different neighborhoods, property types, or construction periods.

In [14]:
# Plot Numerical Variables
fig = px.histogram(df, x='YearBuilt', nbins=30, title='Histogram of YearBuilt')
# Customize the appearance if needed
fig.update_layout(
    xaxis_title='YearBuilt',
    yaxis_title='Counts',
    bargap=0.1,  # Adjust the gap between bars
    title_x=0.5   # Center the title
    )
pyo.iplot(fig)

Key Observations:<br>
Distribution:<br>
The histogram visualizes the frequency of properties built over various years. The x-axis displays the "YearBuilt" values, and the histogram bars show the number of properties constructed in each year.

The histogram may reveal certain periods with peaks, indicating housing booms or periods of significant construction activity.

Histogram Characteristics:<br>
The histogram uses a binning strategy with nbinsx set to 30, meaning the years may be grouped into intervals depending on the overall range of years. However, with a variable like "YearBuilt," each year might be represented by an individual bar.

The plot is unstacked, and each bar represents the number of properties constructed in a specific year or group of years.

Conclusion:<br>
The histogram provides a visual summary of the construction trends over time within the dataset. It can be particularly useful for identifying periods of intense building activity, understanding the age distribution of properties, and spotting any trends related to historical events, economic cycles, or urban development policies. This type of analysis can also be valuable for real estate studies, urban planning, and historical research on housing developments.

In [15]:
# Plot Numerical Variables
fig = px.histogram(df, x='YearRemodAdd', nbins=30, title='Histogram of YearRemodAdd')
# Customize the appearance if needed
fig.update_layout(
    xaxis_title='YearRemodAdd',
    yaxis_title='Counts',
    bargap=0.1,  # Adjust the gap between bars
    title_x=0.5   # Center the title
    )
pyo.iplot(fig)

Key Observations:<br>
Distribution:<br>

The histogram shows the number of properties remodeled or expanded over various years. Each bar represents the count of properties modified in a specific year or group of years.

The histogram may reveal trends in remodeling activities, such as peaks in certain decades when home improvements were particularly popular or necessary.

Histogram Characteristics:<br>
The histogram uses a binning strategy with nbinsx set to 30, which means the years may be grouped into intervals, although with this variable, each year might be represented by its own bar.

The plot is unstacked, and each bar represents the number of properties remodeled or added to in that year.

Conclusion:<br>
This histogram provides a visual overview of when properties in the dataset underwent remodeling or additions. It can help identify periods when renovations were common, possibly reflecting economic conditions, changes in housing regulations, or shifts in design trends. This type of analysis is useful for understanding the lifecycle of properties, planning for future renovations, or analyzing market trends in real estate development.

In [16]:
# Plot Numerical Variables
fig = px.histogram(df, x='BsmtFinSF2', nbins=30, title='Histogram of BsmtFinSF2')
# Customize the appearance if needed
fig.update_layout(
    xaxis_title='BsmtFinSF2',
    yaxis_title='Counts',
    bargap=0.1,  # Adjust the gap between bars
    title_x=0.5   # Center the title
    )
pyo.iplot(fig)

Key Observations:<br>
Distribution:<br>

The histogram shows that the majority of properties have a "BsmtFinSF2" value of 0, indicating that many properties either do not have a second finished area in the basement or that this space is unfinished.

A few properties have non-zero values, representing those that do have a finished second basement area, with square footage values spread across the histogram.

Histogram Characteristics:<br>
The histogram uses 30 bins (nbinsx), which means the data is grouped into 30 intervals along the x-axis. Most of these bins likely contain zero or very few properties, except for the bin representing 0 square feet, which would be the most populated.

The visualization is unstacked, meaning each bar represents the count of properties with a specific range of finished basement square footage.

Conclusion:<br>
This histogram provides a clear visualization of the "BsmtFinSF2" distribution, showing that a large proportion of properties do not have a second finished area in the basement (indicated by the high count at 0 square feet). For properties that do have a finished area, the square footage varies, though it is much less common. This information can be useful for understanding how common it is for properties in the dataset to have a second finished basement area, and it may also help identify trends in property features or the need for remodeling in certain areas.

In [17]:
# Plot Numerical Variables
fig = px.histogram(df, x='TotalBsmtSF', nbins=30, title='Histogram of TotalBsmtSF')
# Customize the appearance if needed
fig.update_layout(
    xaxis_title='TotalBsmtSF',
    yaxis_title='Counts',
    bargap=0.1,  # Adjust the gap between bars
    title_x=0.5   # Center the title
    )
pyo.iplot(fig)

The majority of the homes have basement sizes between approximately 800 and 1500 square feet. This range contains the highest frequency, peaking around 1000-1500 square feet.
This peak suggests that most homes in the dataset have basements of this size, making it the most common basement size range.

In [18]:
# Plot Numerical Variables
fig = px.histogram(df, x='SalePrice', nbins=30, title='Histogram of SalePrice')
# Customize the appearance if needed
fig.update_layout(
    xaxis_title='SalePrice',
    yaxis_title='Counts',
    bargap=0.1,  # Adjust the gap between bars
    title_x=0.5   # Center the title
    )
pyo.iplot(fig)

50% to 60% of homes are priced between $100,000 and $200,000.<br>

70% to 80% of homes are priced below $250,000.<br>

Less than 10% of homes are priced above $300,000.<br>

Outliers and high-value homes priced above $400,000 make up less than 5% of the dataset.<br>

In [19]:
# Explore relationship between SalePrice and other variables
